<a href="https://colab.research.google.com/github/morenourey/CHAT/blob/main/ML_3_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUP ML 3 - PREDICT

# Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler


pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [2]:
# Cargar el modelo guardado si existe
with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
features = model.feature_names_
features

['rev',
 'mou',
 'totmrc',
 'da',
 'ovrmou',
 'ovrrev',
 'vceovr',
 'datovr',
 'roam',
 'change_mou',
 'change_rev',
 'drop_vce',
 'drop_dat',
 'blck_vce',
 'blck_dat',
 'unan_vce',
 'unan_dat',
 'plcd_vce',
 'plcd_dat',
 'recv_vce',
 'recv_sms',
 'comp_vce',
 'comp_dat',
 'custcare',
 'ccrndmou',
 'cc_mou',
 'inonemin',
 'threeway',
 'mou_cvce',
 'mou_cdat',
 'mou_rvce',
 'owylis_vce',
 'mouowylisv',
 'iwylis_vce',
 'mouiwylisv',
 'peak_vce',
 'peak_dat',
 'mou_peav',
 'mou_pead',
 'opk_vce',
 'opk_dat',
 'mou_opkv',
 'mou_opkd',
 'drop_blk',
 'attempt',
 'complete',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'totcalls',
 'totmou',
 'totrev',
 'adjrev',
 'adjmou',
 'adjqty',
 'avgrev',
 'avgmou',
 'avgqty',
 'avg3mou',
 'avg3qty',
 'avg3rev',
 'avg6mou',
 'avg6qty',
 'avg6rev',
 'hnd_price',
 'phones',
 'models',
 'truck',
 'rv',
 'lor',
 'adults',
 'income',
 'numbcars',
 'forgntvl',
 'eqpdays',
 'new_cell_N',
 'new_cell_U',
 'new_cell_Y',
 'crclscod_A',
 'crclsc

# Carga PREDICT dataset

In [4]:
df_predict = pd.read_csv('telecom_churn_PREDICT.csv')

In [5]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 99 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de metricas que varien con la distribución

## Eliminar variables 

In [6]:
# Detectar filas duplicadas
print('Filas duplicadas:', df_predict.duplicated().sum()) 
print('Filas filas con identificador duplicadas:', df_predict['Customer_ID'].duplicated().sum()) 

Filas duplicadas: 0
Filas filas con identificador duplicadas: 0


In [7]:
# Eliminar Columnas identificadoras
df_predict = df_predict.drop(columns=['Customer_ID'])

In [8]:
# Eliminar las mismas columnas eliminadas al entrenar el modelo
to_drop = ['asl_flag' , 'refurb_new' , 'hnd_webcap' , 'ownrent' , 'dwlltype' , 'kid0_2' , 'kid3_5' , 'kid6_10' , 'kid11_15' , 'kid16_17' , 'creditcd']

df_predict = df_predict.drop(columns=to_drop)

In [9]:
# Eliminar filas que tienen todos sus valores vacíos:
df_predict = df_predict.dropna(how='all')

In [10]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 87 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               9777 non-null   float64
 1   mou               9777 non-null   float64
 2   totmrc            9777 non-null   float64
 3   da                9777 non-null   float64
 4   ovrmou            9777 non-null   float64
 5   ovrrev            9777 non-null   float64
 6   vceovr            9777 non-null   float64
 7   datovr            9777 non-null   float64
 8   roam              9777 non-null   float64
 9   change_mou        9704 non-null   float64
 10  change_rev        9704 non-null   float64
 11  drop_vce          10000 non-null  float64
 12  drop_dat          10000 non-null  float64
 13  blck_vce          10000 non-null  float64
 14  blck_dat          10000 non-null  float64
 15  unan_vce          10000 non-null  float64
 16  unan_dat          10000 non-null  float64

## Imputacion de nulos

In [11]:
df_predict_num = df_predict.select_dtypes(include=['int64', 'float64'])
df_predict_num.isna().sum().sort_values(ascending=False)

numbcars    5798
lor         3688
income      3240
adults      2986
avg6rev     1432
            ... 
mou_peav       0
opk_vce        0
opk_dat        0
mou_opkv       0
eqpdays        0
Length: 77, dtype: int64

### imputar nulos a columnas numericas

In [12]:
def fill_num_na(df_churn_num, columnas_rellenar):
    for col in columnas_rellenar:
        # Proporciones calculadas
        value_counts = df_churn_num[col].value_counts(normalize=True)

        # Lista de valores para rellenar
        fill_values = np.random.choice(value_counts.index, size=df_churn_num[col].isna().sum(), p=value_counts.values)

        # Rellenando los nulos
        df_churn_num.loc[df_churn_num[col].isna(), col] = fill_values

    return df_churn_num

# Columnas a rellenar
columnas_rellenar = ['lor', 'adults', 'income', 'numbcars']

# Rellenando
df_predict = fill_num_na(df_predict, columnas_rellenar)

### imputar nulos a columnas categoricas

In [13]:
cat_columns = df_predict.select_dtypes(include=['object'])

# Rellenar nulos de forma proporcional con los valores de la columna
for col in df_predict:
    if(col not in cat_columns.columns): continue

    if(df_predict[col].value_counts().index.size == 1):
        # En caso de que la columna solo tenga un mismo valor, se rellena con 'None'
        df_predict[col] = df_predict[col].fillna('None')
    else:
        # Proporciones calculadas
        value_counts = df_predict[col].value_counts(normalize=True)

        # Lista de valores para rellenar
        fill_values = np.random.choice(value_counts.index, size=df_predict[col].isna().sum(), p=value_counts.values)

        # Rellenando los nulos
        df_predict.loc[df_predict[col].isna(), col] = fill_values

### dropear al resto

In [14]:
df_predict.dropna(inplace=True)

In [15]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7968 entries, 0 to 9993
Data columns (total 87 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rev               7968 non-null   float64
 1   mou               7968 non-null   float64
 2   totmrc            7968 non-null   float64
 3   da                7968 non-null   float64
 4   ovrmou            7968 non-null   float64
 5   ovrrev            7968 non-null   float64
 6   vceovr            7968 non-null   float64
 7   datovr            7968 non-null   float64
 8   roam              7968 non-null   float64
 9   change_mou        7968 non-null   float64
 10  change_rev        7968 non-null   float64
 11  drop_vce          7968 non-null   float64
 12  drop_dat          7968 non-null   float64
 13  blck_vce          7968 non-null   float64
 14  blck_dat          7968 non-null   float64
 15  unan_vce          7968 non-null   float64
 16  unan_dat          7968 non-null   float64
 17  

In [16]:
df_predict['infobase'].value_counts()

infobase
M    7745
N     223
Name: count, dtype: int64

In [17]:
# en train se transformaron todos los datos de la columna infobase que no fuesen 'M' a 'None'
df_predict['infobase'] = df_predict['infobase'].apply(lambda x: x if x == 'M' else 'None')

In [18]:
df_predict['infobase'].value_counts()

infobase
M       7745
None     223
Name: count, dtype: int64

## reducir la cardinalidad

In [19]:
# Agrupar valores en categoria 'otros' para columnas igual que las obtenidas en el ML_1
values_crclscod = ['A', 'AA', 'B', 'BA', 'C', 'CA', 'DA', 'EA', 'ZA']
values_dwllsize = ['A', 'B', 'C', 'D', 'J', 'K', 'N', 'O']
values_ethnic = ['B', 'F', 'G', 'H', 'I', 'J', 'N', 'O', 'R', 'S', 'U', 'Z']
columnas_agrupar = {'crclscod': values_crclscod,'dwllsize': values_dwllsize, 'ethnic': values_ethnic }

for columna in columnas_agrupar:
  valores_filtrados = columnas_agrupar[columna]

  df_predict[columna] = df_predict[columna].apply(lambda x: x if x in valores_filtrados else 'otros')

## Encodings

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [20]:
columns_to_encode = ['new_cell' , 'crclscod' , 'prizm_social_one' , 'area' , 'dualband' , 'marital' , 'HHstatin' , 'dwllsize' , 'ethnic' , 'infobase']

encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(df_predict[columns_to_encode])

In [21]:
# Converter os dados codificados para um DataFrame
encoded_columns = encoder.get_feature_names_out(columns_to_encode)
one_hot_encoded_df  = pd.DataFrame(encoded_categorical_data, columns=encoded_columns)

In [22]:
df_predict = pd.concat([df_predict, one_hot_encoded_df], axis=1)

In [23]:
df_predict = df_predict.drop(columns=columns_to_encode)

In [24]:
# drop columnas de alta correlacion

In [25]:
df_predict.columns.to_list()

['rev',
 'mou',
 'totmrc',
 'da',
 'ovrmou',
 'ovrrev',
 'vceovr',
 'datovr',
 'roam',
 'change_mou',
 'change_rev',
 'drop_vce',
 'drop_dat',
 'blck_vce',
 'blck_dat',
 'unan_vce',
 'unan_dat',
 'plcd_vce',
 'plcd_dat',
 'recv_vce',
 'recv_sms',
 'comp_vce',
 'comp_dat',
 'custcare',
 'ccrndmou',
 'cc_mou',
 'inonemin',
 'threeway',
 'mou_cvce',
 'mou_cdat',
 'mou_rvce',
 'owylis_vce',
 'mouowylisv',
 'iwylis_vce',
 'mouiwylisv',
 'peak_vce',
 'peak_dat',
 'mou_peav',
 'mou_pead',
 'opk_vce',
 'opk_dat',
 'mou_opkv',
 'mou_opkd',
 'drop_blk',
 'attempt',
 'complete',
 'callfwdv',
 'callwait',
 'months',
 'uniqsubs',
 'actvsubs',
 'totcalls',
 'totmou',
 'totrev',
 'adjrev',
 'adjmou',
 'adjqty',
 'avgrev',
 'avgmou',
 'avgqty',
 'avg3mou',
 'avg3qty',
 'avg3rev',
 'avg6mou',
 'avg6qty',
 'avg6rev',
 'hnd_price',
 'phones',
 'models',
 'truck',
 'rv',
 'lor',
 'adults',
 'income',
 'numbcars',
 'forgntvl',
 'eqpdays',
 'new_cell_N',
 'new_cell_U',
 'new_cell_Y',
 'crclscod_A',
 'crclsc

# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

In [26]:
# MinMaxScaler para las numericas

df_predict.dropna(inplace=True)
df_churn_num = df_predict.select_dtypes(include=['int64', 'float64'])
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_churn_num)
df_churn_num = pd.DataFrame(X_scaled, columns=df_churn_num.columns)
df_churn_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6225 entries, 0 to 6224
Columns: 152 entries, rev to infobase_None
dtypes: float64(152)
memory usage: 7.2 MB


In [27]:
for col in df_churn_num:
    df_predict[col] = df_churn_num[col]

## Reordenar

In [28]:
# Get the columns of df_train
df_train = pd.read_pickle("final_df.pkl")
train_columns = df_train.columns.to_list()
train_columns.remove('churn')

# Reorder df_predict columns to match df_train
df_predict = df_predict[train_columns]

# PREDICT

* predict() y predict_proba()

In [29]:
# leer pickle del modelo
with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [30]:
y_pred = best_model.predict(df_predict)
y_pred_prob = best_model.predict_proba(df_predict)

y_pred


array([False, False, False, ...,  True,  True,  True])

In [31]:
y_pred_prob

array([[0.57386726, 0.42613274],
       [0.51348834, 0.48651166],
       [0.73417797, 0.26582203],
       ...,
       [0.29432517, 0.70567483],
       [0.29432517, 0.70567483],
       [0.29432517, 0.70567483]])

In [32]:
# convertir las probabilidades en 1-dimensional id, quedandonos con el mayor
max_prob = np.max(y_pred_prob, axis=1)

In [33]:
max_prob

array([0.57386726, 0.51348834, 0.73417797, ..., 0.70567483, 0.70567483,
       0.70567483])

In [34]:
# Convertir y_pred a una Series de Pandas
y_pred_series = pd.Series(y_pred, name='y_pred'),

y_prob_series = pd.Series(max_prob, name='y_prob')

In [37]:
y_prob_series

0       0.573867
1       0.513488
2       0.734178
3       0.570385
4       0.560622
          ...   
6220    0.705675
6221    0.705675
6222    0.705675
6223    0.705675
6224    0.705675
Name: y_prob, Length: 6225, dtype: float64

# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [40]:
df_predicted = pd.concat([df_predict, y_pred_series, y_prob_series], axis=1)

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [ ]:
df_predicted.to_pickle("df_predicted.pkl")